# List out Deployments and Quota
This will create a list of deployments from Azure OpenAI with the usage (Quota). Thanks [@ssemyan](https://github.com/ssemyan) for the start of this

In [31]:
import requests,json
from azure.identity import DefaultAzureCredential
import csv


# Use your credentials to access the subscriptions
api = "https://management.azure.com/.default"
credential = DefaultAzureCredential()
access_token = credential.get_token(api) 
header = {'Authorization': f'Bearer {access_token.token}'}

In [32]:
#Get a list of subscriptions
subs = requests.get('https://management.azure.com/subscriptions?api-version=2022-12-01',headers=header).json()

# Quota and Deployment
Quota is by Subscription and Region and Model so notice how this grabs that quota per Subscription

> This does tend to timeout or connection error so you might see that too

> NOTE: Quota is not directly tied to deployments so you really need to look at both separately

In [ ]:
items=[]
quotas=[]
for sub in subs['value']:
    subId=sub['subscriptionId']

    locations = requests.get(f"https://management.azure.com/subscriptions/{subId}/locations?api-version=2022-12-01",headers=header).json()

    quota={}
    # Get the quota by the sub and locations
    for loc in locations['value']:
        location=loc['name']
        if location in quota:
            continue
        else:
            quota[location]={}
            try:
                usages = requests.get(f"https://management.azure.com/subscriptions/{subId}/providers/Microsoft.CognitiveServices/locations/{location}/usages?api-version=2023-05-01",headers=header,timeout=5).json()
                for usage in usages['value']:
                    if usage['name']['value'].startswith("OpenAI.Standard."):
                        quota_name = usage['name']['value'].replace("OpenAI.Standard.", "").lower()
                        quota[location][quota_name] = usage['limit']       
                        quotas.append({'subscriptionId':subId,'location':location,'quota':quota_name,'limit':usage['limit'],'currentValue':usage['currentValue']})     
            except:
                # A lot of locations don't have quotas
                # print(f"Error getting quota for {subId} in {location}")
                continue
    
    # Get Resource Groups
    uri=f"https://management.azure.com/subscriptions/{subId}/resourceGroups?api-version=2022-12-01"
    rgs = requests.get(f"https://management.azure.com/subscriptions/{subId}/resourceGroups?api-version=2022-12-01",headers=header).json()
    for rg in rgs['value']:
        resourceGroupName=rg['name']
        usage_lookup = {}

        print("rg=",rg['name'])

        # get a list of accounts
        accounts = requests.get(f"https://management.azure.com/subscriptions/{subId}/resourceGroups/{resourceGroupName}/providers/Microsoft.CognitiveServices/accounts?api-version=2023-05-01",headers=header).json()
        for account in accounts['value']:
            accountName=account['name']
            # Actual resource might be in a different location than the resource group
            location=account['location']

            print(f"Account: {accountName} Location: {location}")
            
            deployments = requests.get(f"https://management.azure.com/subscriptions/{subId}/resourceGroups/{resourceGroupName}/providers/Microsoft.CognitiveServices/accounts/{accountName}/deployments?api-version=2023-05-01",headers=header).json()
            for deployment in deployments['value']:

                name=deployment['name']
                model=deployment['properties']['model']['name']
                version=deployment['properties']['model']['version']
                for rl in deployment['properties']['rateLimits']:
                    if rl['key'] == 'token':
                        tpm = rl['count']

                qt=None
                if location in quota:
                    if model in quota[location]:
                        qt = quota[location][model]*1000
                items.append({'subscriptionId':subId,'resourceGroupName':resourceGroupName,'accountName':accountName,'deploymentName':name,'model':model,'tpm':tpm,'quota':qt,'location':location,'version':version})


In [37]:
# Output this to a tab delimited file
fieldnames = ['subscriptionId','resourceGroupName','accountName','deploymentName','model','tpm','quota','location','version']
with open('deployments.csv', 'w') as file:
    writer = csv.DictWriter(file, fieldnames=fieldnames,delimiter='\t')
    writer.writeheader()
    writer.writerows(items)


In [34]:
# Output quotas to a tab delimited file
fieldnames = ['subscriptionId','location','quota','limit','currentValue']
with open('quotas.csv', 'w') as file:
    writer = csv.DictWriter(file, fieldnames=fieldnames,delimiter='\t')
    writer.writeheader()
    writer.writerows(quotas)



# Combine and conquer
Take the output from the two files and use some `=SUMIFS()` logic to show quota used versus what is still available

> Note: You do get `currentValue` in the Quota/Usages so there is a quick place to look to see how much is being used. Comparing Deployments to Usage is not straight forward, wish it was 